## Corona Setup

In [5]:
#Data visualization

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#Audio Analysis
import glob
import IPython
import librosa
import librosa.display
import tensorflow as tf
from tensorflow import keras

#path
import os

#

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
#import meta data

df_meta = pd.read_csv('./CoronaHack-Respiratory-Sound-Dataset/Corona-Hack-Respiratory-Sound-Metadata.csv')
df_meta.info(), df_meta.shape


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1397 entries, 0 to 1396
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   USER_ID                 1397 non-null   object 
 1   COUNTRY                 1397 non-null   object 
 2   AGE                     1397 non-null   int64  
 3   COVID_STATUS            1396 non-null   object 
 4   ENGLISH_PROFICIENCY     1397 non-null   object 
 5   GENDER                  1397 non-null   object 
 6   COUNTY_RO_STATE         1397 non-null   object 
 7   CITY_LOCALITY           1228 non-null   object 
 8   Diabetes                1397 non-null   int64  
 9   Asthma                  1397 non-null   int64  
 10  Smoker                  1397 non-null   int64  
 11  Hypertension            1397 non-null   int64  
 12  Fever                   1397 non-null   int64  
 13  Returning_User          1397 non-null   int64  
 14  Using_Mask              1397 non-null   

(None, (1397, 37))

In [26]:
#Use path info from csv file to load data 

#split COVID STATUS to get labels
df_meta['split'] = df_meta['COVID_STATUS'].str.split('_').str.get(0)
#Check for NA
df_meta.loc[:,'counting-normal'].isna().sum()
df_meta.loc[:,'split'].value_counts()

#Generate a dict to categorize split column
cat_dict = {'healthy':0,'no':0,'resp':0,'recovered':0,'positive':1}

#map cat_dict to split column 
df_meta.loc[:,'split'] =  df_meta.loc[:,'split'].map(cat_dict)
df_meta2 = df_meta.dropna(subset=['split'])
df_meta2.loc[:,'split'] = df_meta2.loc[:,'split'].astype('int32')


#Extract positive USER ID
df_meta_positives = df_meta[df_meta['split'] == 1]
df_meta_negatives = df_meta[df_meta['split'] == 0]

positives = list(df_meta_positives['USER_ID'])
negatives = list(df_meta_negatives['USER_ID'])
len(positives),len(negatives)
#positives

C:\Users\paulg\.conda\envs\corona\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


(56, 1340)

In [27]:
df_meta2.head()

,USER_ID,COUNTRY,AGE,COVID_STATUS,ENGLISH_PROFICIENCY,GENDER,COUNTY_RO_STATE,CITY_LOCALITY,Diabetes,Asthma,...,breathing-deep,breathing-shallow,cough-heavy,cough-shallow,counting-fast,counting-normal,vowel-a,vowel-e,vowel-o,split
0,vK2bLRNzllXNeyOMudnNSL5cfpG2,India,24,healthy,Y,M,Karnataka,Bangalore,0,0,...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,0
1,bjA2KpSxneNskrLBeqi4bqoTDQl2,India,72,healthy,Y,M,Maharashtra,Thane,0,0,...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,0
2,FSzobvJqOXf0rI6X05cHqOiU9Mu2,India,54,healthy,Y,M,Maharashtra,Thane West,0,0,...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,0
3,EqDWckxbsETyHUeBLQ8jLtxlhir2,India,31,healthy,Y,M,Karnataka,Bangalore,0,0,...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,/data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...,0
4,FGRDO4IBbAejR0WHD5YbkXTCasg2,India,26,healthy,Y,M,Haryana,gurgaon,0,0,...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,/data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...,0


In [6]:
import cv2
def preprocess_other(sample):
  image_target_height, image_target_width = 64, 64 #setting up the shape of sample
  audio_binary = tf.io.read_file(sample) #read-in the sample 
  audio, rate = tf.audio.decode_wav(audio_binary, desired_channels=1) #getting the audio and rate
  #label = sample['label']

  def py_preprocess_audio(audio):
      audio = audio.numpy().astype('float32')

      spectogram = librosa.feature.melspectrogram(
        y=audio, n_fft=1024,  n_mels=64, hop_length=64, sr=8000, fmax=2000 #n_fft = window size, n_mels = frequency bins, hop_lenghth =jump to the right , sr = sound rate, fmax = 
      ) 

      spectogram /= np.max(spectogram) #devide by np.max(audio)
      spectogram = cv2.resize(spectogram, dsize=(image_target_height, image_target_width))
      spectogram = np.expand_dims(spectogram, axis=-1)
      return spectogram

  spectogram = tf.py_function(py_preprocess_audio, [audio], tf.float32)
  spectogram.set_shape((image_target_height, image_target_width, 1))

  return spectogram#, label

In [35]:

#test
path = './CoronaHack-Respiratory-Sound-Dataset' + df_meta.loc[16,'counting-normal']
#print(path)
def pr(sample):
    image_target_height, image_target_width = 64, 64 #setting up the shape of sample
    audio_binary = tf.io.read_file(sample) #read-in the sample 
    audio, rate = tf.audio.decode_wav(audio_binary, desired_channels=1) #getting the audio and rate
    #audio = audio[:,0]
    #print(audio.shape)
    return(audio)

print(pr(path))

tf.Tensor(
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]], shape=(864256, 1), dtype=float32)


In [7]:
path_counnting_normal = df_meta2['counting-normal'].tolist()
path_counnting_normal = ['./CoronaHack-Respiratory-Sound-Dataset' + dir_name for dir_name in path_counnting_normal]
#labels = tf.convert_to_tensor()



# Convert to Tensor
sound_paths_counnting_normal = tf.convert_to_tensor(path_counnting_normal, dtype=tf.string)
# Build a TF Queue, shuffle data
#sound,labels = tf.data.Dataset.from_tensor_slices([sound_paths_counnting_normal, labels])
#make tensor slices and map in batches of 32
sound = tf.data.Dataset.from_tensor_slices(sound_paths_counnting_normal)
x = sound.map(lambda sample: preprocess_other(sample)).batch(32)
x

<BatchDataset shapes: (None, 64, 64, 1), types: tf.float32>

In [50]:
names = ['counting-normal','counting-fast','breathing-deep','breathing-shallow','cough-heavy','cough-shallow','vowel-a','vowel-e','vowel-o']
input_dic = {}
for index,name in enumerate(names):
    #print(index,name)
    path_list = df_meta2[name].tolist()
    #print(path_list[:10])
    path_name = ['./CoronaHack-Respiratory-Sound-Dataset'  + str(dir_name for dir_name in path_list)]
    sound_paths_tensor = tf.convert_to_tensor(path_name, dtype=tf.string)
    sound = tf.data.Dataset.from_tensor_slices(sound_paths_tensor)
    input_dic['x_{}'.format(index)] = sound.map(lambda sample: preprocess_other(sample))


name_label = 'split'
path_label = df_meta2[name_label].tolist()
label_paths_tensor = tf.convert_to_tensor(path_label, dtype=tf.int16)
y = tf.data.Dataset.from_tensor_slices(label_paths_tensor)


In [55]:
names_input = ['counting-normal','counting-fast','breathing-deep','breathing-shallow','cough-heavy','cough-shallow','vowel-a','vowel-e','vowel-o']
name_label = 'split'

def create_input_label(df=df_meta2,names=names_input,name_label=name_label):
    input_dic = {}
    for index,name in enumerate(names):
        #print(index,name)
        path_list = df[name].tolist()
        #print(path_list[:10])
        path_name = ['./CoronaHack-Respiratory-Sound-Dataset'  + str(dir_name for dir_name in path_list)]
        sound_paths_tensor = tf.convert_to_tensor(path_name, dtype=tf.string)
        input_dic['x_{}'.format(index)] = sound.map(lambda sample: preprocess_other(sample))


    path_label = df_meta2[name_label].tolist()
    y = tf.convert_to_tensor(path_label, dtype=tf.int16)

    return input_dic,y

In [58]:
def _input_fn():
  input_dict,labels = create_input_label(df=df_meta2,names=names_input,name_label=name_label)
  dataset = tf.data.Dataset.from_tensor_slices((input_dict, labels))
  dataset = dataset.batch(32, drop_remainder=True)
  return dataset

In [51]:
from tensorflow.keras import models, layers

model = models.Sequential()

model.add(layers.Conv2D(4, (3,3), activation='relu', padding='same', input_shape=(64,64,1))) #3,3 is the pixel window for screen and is a common standard, 4 is also a parameter
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(8, (3,3),activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(8, (3,3),activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(16, (3,3),activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(2, activation='sigmoid'))

model.summary()

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    #metrics=['accuracy']
)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 64, 64, 4)         40        
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 32, 32, 4)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 8)         296       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 8)         584       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 8, 8, 8)           0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 8, 8, 16)         

In [60]:


history_list = {}

history = model.fit(
     _input_fn(),
    epochs = 20,
    #batch_size=32

)

history_list['base'] = history

ValueError: Unbatching a dataset is only supported for rank >= 1

In [64]:
input1 = keras.layers.Input(shape=(64,64,1, ))
input2 = keras.layers.Input(shape=(64,64,1,))
merged = keras.layers.Concatenate(axis=1)([input1, input2])

cv2_first = layers.Conv2D(4, (3,3), activation='relu', padding='same',  input_dim=2,)(merged) #3,3 is the pixel window for screen and is a common standard, 4 is also a parameter
pooling_1 = layers.MaxPooling2D((2,2))(cv2_first)


flatten = layers.Flatten()(pooling_1)

densse_1 = layers.Dense(64, activation='relu')(flatten)

output = layers.Dense(2, activation='sigmoid')(densse_1)

model_api = keras.models.Model(inputs=[input1, input2], outputs=output)

model_api.summary()

model_api.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    #metrics=['accuracy']
)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 128, 64, 1)   0           input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 128, 64, 4)   40          concatenate_3[0][0]        

In [68]:
input_dict,labels = create_input_label(df=df_meta2,names=names_input,name_label=name_label)

model_api.fit([input_dic['x_0'], input_dic['x_1']],labels, batch_size=16, epochs=100)


ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>"}), <class 'tensorflow.python.framework.ops.EagerTensor'>

In [71]:
labels

<tf.Tensor: shape=(1396,), dtype=int16, numpy=array([0, 0, 0, ..., 1, 0, 0], dtype=int16)>

In [36]:
def wav_to_mel(file_path):
    image_target_height, image_target_width = 64, 64
    audio_binary = tf.io.read_file(file_path)
    audio, rate = tf.audio.decode_wav(audio_binary, desired_channels=1)
    #waveform = tf.squeeze(audio, axis=-1)
    audio = audio[:,0]
    print(audio.shape)
    position = tfio.audio.trim(audio, axis=0, epsilon=0.1)
    print(position[0])
    start = position[0]
    end = position[1]
    audio= audio[start:end]

    # Normalize audio data
    audio = tf.cast(audio, tf.float32) / 32768.0  # Max int for audio data
    # Create the spectogram from audio data
    spectrogram = tfio.audio.spectrogram(
        audio, nfft=1024, window=128, stride=64
    )
    # Turn spectrogram into mel spectrogram
    spectrogram = tfio.audio.melscale(
        spectrogram, rate=rate, mels=64, fmin=0, fmax=2000
    )

    #spectrogram /= np.max(audio)
    spectrogram /= tf.math.reduce_max(spectrogram) # Normalize
    spectrogram = tf.expand_dims(spectrogram, axis=-1) # 2D -> 3D
    spectrogram = tf.image.resize(spectrogram, (image_target_height, image_target_width)) # Resize the picture
    spectrogram = tf.transpose(spectrogram, perm=(1, 0, 2)) # Swap the first two axis
    spectrogram = spectrogram[::-1, :, :] # Flip the first axis (frequency)

    #display(Audio(audio, rate=8000))
   
    return spectrogram


In [36]:
import tensorflow_io as tfio
from IPython.display import Audio, display

sounds = ['counting-fast','counting-normal']
spects = {}
# initialize with empty list
for i in range(len(df_meta2)):
        spects[i] = []
# create spect for each sound
for sound in sounds:
    for i in range(len(df_meta2)):
        path = './CoronaHack-Respiratory-Sound-Dataset' + df_meta2.loc[i,sound]
        spects[i].append(preprocess_other(path))

#spects   
print(spects)


UnknownError: ParameterError: Invalid shape for monophonic audio: ndim=2, shape=(196608, 1)
Traceback (most recent call last):

  File "C:\Users\paulg\.conda\envs\corona\lib\site-packages\tensorflow\python\ops\script_ops.py", line 247, in __call__
    return func(device, token, args)

  File "C:\Users\paulg\.conda\envs\corona\lib\site-packages\tensorflow\python\ops\script_ops.py", line 135, in __call__
    ret = self._func(*args)

  File "C:\Users\paulg\.conda\envs\corona\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 645, in wrapper
    return func(*args, **kwargs)

  File "<ipython-input-10-867061280308>", line 12, in py_preprocess_audio
    y=audio, n_fft=1024,  n_mels=64, hop_length=64, sr=8000, fmax=2000 #n_fft = window size, n_mels = frequency bins, hop_lenghth =jump to the right , sr = sound rate, fmax =

  File "C:\Users\paulg\.conda\envs\corona\lib\site-packages\librosa\feature\spectral.py", line 2004, in melspectrogram
    pad_mode=pad_mode,

  File "C:\User

In [ ]:
 !pip install pydub tensorflow_io

In [ ]:
#load data

def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    sound = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) 
                      if f.endswith(".wav")]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    return images, labels

ROOT_PATH = "/your/root/path"
train_data_directory = os.path.join(ROOT_PATH, "./CoronaHack-Respiratory-Sound-Dataset/data/train")
test_data_directory = os.path.join(ROOT_PATH, "./CoronaHack-Respiratory-Sound-Dataset/data/test")

images, labels = load_data(train_data_directory)